In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#carregar os dados

train = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')

teste = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')

In [ ]:
#verificando df treino

train.info()

In [ ]:
#Verificando o df de teste
teste.info()

In [ ]:
train.head()

In [ ]:
#Transformando o dataframe original na coluna count
#vamos usar escala logaritimica

train['count'] = np.log(train['count'])


In [ ]:
train = train.append(teste)

In [ ]:
train.head()



In [ ]:
train['datetime'] = pd.to_datetime(train['datetime'])

In [ ]:
#crindo nova coluna usando data e hora

train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train['dayofweek'] = train['datetime'].dt.dayofweek



In [ ]:
train.head()

In [ ]:
#separando o df de treino e teste

#primeiro teste

teste = train[train['count'].isnull()]

In [ ]:
teste.shape

In [ ]:
#separando o df de treino e teste

#segundo treino

treino = train[~train['count'].isnull()]

In [ ]:
treino.shape

In [ ]:
#Separando o df de treino em treino/validação (def = 75/25)

from sklearn.model_selection import train_test_split

treino, validacao = train_test_split(treino, random_state=42)

In [ ]:
print(treino.shape)
treino.head()

In [ ]:
print(validacao.shape)
validacao.head()


In [ ]:
#selecionando colunas

#colunas nao usadas:

nao_usadas = ['datetime', 'casual', 'registered', 'count']

#colunas usadas:

#lista_usadas = ['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity','windspeed']

usadas = [c for c in treino if c not in nao_usadas]

usadas

# Arvore de decisao

In [ ]:
#importando

from sklearn.tree import DecisionTreeRegressor

In [ ]:
#instanciando objeto de decision tree

ad = DecisionTreeRegressor(random_state=42)

In [ ]:
#treinando o modelo
#informar as colunas de entrada e a coluna de resposta (target)

ad.fit(treino[usadas], treino['count'])

In [ ]:
#prever os dados de validação

previsao = ad.predict(validacao[usadas])

In [ ]:
previsao

In [ ]:
validacao['count']

In [ ]:
#usando a metrica para validar os dados

from sklearn.metrics import mean_squared_error

In [ ]:
#Calculando a métrica
mean_squared_error(validacao['count'], previsao)**(1/2)


#1.5756091264794638
#erro alto... quanto mais perto de 0 melhor

13/11 - Aula 2



In [ ]:
#importar o modelo Random Forest

from sklearn.ensemble import RandomForestRegressor

In [ ]:
#instanciar o modelo
rf = RandomForestRegressor(random_state=42, n_jobs=1)

In [ ]:
#treinando o modelo
rf.fit(treino[usadas], treino['count'])

In [ ]:
#Fazendo previsões em cima dos dados de validação

preds = rf.predict(validacao[usadas])

In [ ]:
preds

In [ ]:
#verificando o real

validacao['count'].head(3)

In [ ]:
#verificando o modelo com relação a métrica

#importando a métrica

from sklearn.metrics import mean_squared_error

In [ ]:
#aplicando a métrica
mean_squared_error(validacao['count'], preds) ** (1/2)
#0.348204 do professor

In [ ]:
#vamos prever com base nos dados de treino
# como o modelo se comporta prevendo em cima de dados conhecidos
# o modelo ja esta treinado

treino_preds = rf.predict(treino[usadas])

mean_squared_error(treino['count'], treino_preds) ** (1/2)

In [ ]:
#Gerando as previsões para envio ao Kaggle
teste['count'] = np.exp(rf.predict(teste[usadas]))

In [ ]:
#visualizando o arquivo para envio
teste[['datetime','count']].head()

In [ ]:
#gerando csv
teste[['datetime','count']].to_csv('rf.csv', index=False)